In [11]:
import roifile
from roifile import ImagejRoi
import numpy as np
import cv2
from PIL import Image

from pathlib import Path

import matplotlib.pyplot as plt

In [12]:
# Read single ROI file
def read_imagej_roi(roi_path):
    """Read a single ImageJ .roi file"""
    roi = ImagejRoi.fromfile(roi_path)
    
    print(f"ROI Type: {roi.roitype}")
    print(f"ROI Name: {roi.name}")
    print(f"Coordinates shape: {roi.coordinates().shape if hasattr(roi, 'coordinates') else 'N/A'}")
    
    # Get coordinates
    coords = roi.coordinates()
    return coords, roi

# Convert ROIs to label map
def rois_to_label_map(roi_path, img_height, img_width, category_mapping=None):
    """
    Convert ImageJ ROI file(s) to label map
    
    Args:
        roi_path: Path to .roi or .zip file
        img_height: Image height
        img_width: Image width
        category_mapping: Dict mapping ROI names to category IDs
    """
    # Read ROIs
    rois = ImagejRoi.fromfile(roi_path)
    if not isinstance(rois, list):
        rois = [rois]
    
    # Initialize label map
    label_map = np.zeros((img_height, img_width), dtype=np.uint8)
    
    # Auto-generate category mapping if not provided
    if category_mapping is None:
        unique_names = set(roi.name for roi in rois)
        category_mapping = {name: idx + 1 for idx, name in enumerate(sorted(unique_names))}
        print("Auto-generated mapping:", category_mapping)
    
    # Draw each ROI
    for roi in rois:
        coords = roi.coordinates()
        category_id = category_mapping.get(roi.name, 1)
        
        # Convert to integer coordinates
        points = coords.astype(np.int32)
        
        # Fill polygon
        cv2.fillPoly(label_map, [points], category_id)
        print(f"Drew ROI '{roi.name}' with {len(points)} points, category ID: {category_id}")
    
    return label_map, category_mapping

In [21]:
img_height = 504  # CHANGE to your image height
img_width = 663 

file_roi = Path("0040/RoiSet_Zelle1/0212-0230-0195.roi")


coords, roi = read_imagej_roi(file_roi)

print(coords[:,0].min(),coords[:,0].max())
print(coords[:,1].min(),coords[:,1].max())

label_map, category_mapping = rois_to_label_map(file_roi, img_height, img_width, category_mapping=None)

ROI Type: 7
ROI Name: 0212-0230-0195
Coordinates shape: (38, 2)
192.0 197.33333
227.0 231.66666
Auto-generated mapping: {'0212-0230-0195': 1}
Drew ROI '0212-0230-0195' with 38 points, category ID: 1


In [17]:
root = Path("0040")

# Find all subfolders containing "roi" and "set" in their name (case-insensitive)
roi_set_folders = [
    folder for folder in root.iterdir() 
    if folder.is_dir() and "roi" in folder.name.lower() and "set" in folder.name.lower()
]

print(f"Found {len(roi_set_folders)} RoiSet folders:")
for folder in sorted(roi_set_folders):
    print(f"  - {folder.name}")

# Get all ROI files from all RoiSet folders
all_roi_files = []
for folder in roi_set_folders:
    roi_files = sorted(list(folder.glob("*.roi")))
    all_roi_files.extend(roi_files)
    print(f"\n{folder.name}: {len(roi_files)} ROI files")

print(f"\nTotal ROI files across all folders: {len(all_roi_files)}")

Found 14 RoiSet folders:
  - RoiSet_Zelle1
  - RoiSet_Zelle10
  - RoiSet_Zelle11
  - RoiSet_Zelle12
  - RoiSet_Zelle13
  - RoiSet_Zelle14
  - RoiSet_Zelle2
  - RoiSet_Zelle3
  - RoiSet_Zelle4
  - RoiSet_Zelle5
  - RoiSet_Zelle6
  - RoiSet_Zelle7
  - RoiSet_Zelle8
  - RoiSet_Zelle9

RoiSet_Zelle1: 329 ROI files

RoiSet_Zelle10: 338 ROI files

RoiSet_Zelle11: 435 ROI files

RoiSet_Zelle12: 433 ROI files

RoiSet_Zelle13: 344 ROI files

RoiSet_Zelle14: 386 ROI files

RoiSet_Zelle2: 352 ROI files

RoiSet_Zelle3: 298 ROI files

RoiSet_Zelle4: 345 ROI files

RoiSet_Zelle5: 268 ROI files

RoiSet_Zelle6: 292 ROI files

RoiSet_Zelle7: 428 ROI files

RoiSet_Zelle8: 255 ROI files

RoiSet_Zelle9: 316 ROI files

Total ROI files across all folders: 4819


In [23]:
from roifile import ImagejRoi
from pathlib import Path
import re

root_roi = Path("0040/RoiSet_Zelle1")
roi_files = sorted(list(root_roi.glob("*.roi")))

print(f"Found {len(roi_files)} ROI files\n")

# Parse filenames to extract the three numbers
def parse_roi_filename(filename):
    """Extract three numbers from filename like '0212-0230-0195.roi'"""
    match = re.match(r'(\d+)-(\d+)-(\d+)\.roi', filename)
    if match:
        return tuple(map(int, match.groups()))
    return None

# Check ROI properties
print("=== Checking first few ROI files ===")
for i, roi_file in enumerate(roi_files[:5]):
    roi = ImagejRoi.fromfile(roi_file)
    coords = roi.coordinates()
    filename_nums = parse_roi_filename(roi_file.name)
    
    print(f"\n{roi_file.name}:")
    print(f"  Filename numbers: {filename_nums}")
    print(f"  ROI position (z/c/t): {roi.position}")  # ImageJ stack position
    print(f"  ROI type: {roi.roitype}")
    print(f"  Coordinate bounds: x=[{coords[:, 0].min():.0f}, {coords[:, 0].max():.0f}], "
          f"y=[{coords[:, 1].min():.0f}, {coords[:, 1].max():.0f}]")
    
    # Check if filename numbers match ROI properties
    if hasattr(roi, 'z_position'):
        print(f"  Z position in ROI: {roi.z_position}")
    if hasattr(roi, 'c_position'):
        print(f"  Channel position: {roi.c_position}")
    if hasattr(roi, 't_position'):
        print(f"  Time position: {roi.t_position}")

# Analyze all filenames to see pattern
print("\n\n=== Analyzing all filename patterns ===")
all_nums = [parse_roi_filename(f.name) for f in roi_files]
all_nums = [n for n in all_nums if n is not None]

if all_nums:
    num1, num2, num3 = zip(*all_nums)
    
    print(f"First number range: {min(num1)} - {max(num1)} (unique: {len(set(num1))})")
    print(f"Second number range: {min(num2)} - {max(num2)} (unique: {len(set(num2))})")
    print(f"Third number range: {min(num3)} - {max(num3)} (unique: {len(set(num3))})")
    
    # Check if they increment by 1 (suggesting z-slices)
    num1_sorted = sorted(set(num1))
    num2_sorted = sorted(set(num2))
    num3_sorted = sorted(set(num3))
    
    def is_sequential(nums):
        if len(nums) <= 1:
            return False
        diffs = [nums[i+1] - nums[i] for i in range(len(nums)-1)]
        return all(d == 1 for d in diffs) or all(d == diffs[0] for d in diffs)
    
    print(f"\nFirst number sequential: {is_sequential(num1_sorted)}")
    print(f"Second number sequential: {is_sequential(num2_sorted)}")
    print(f"Third number sequential: {is_sequential(num3_sorted)}")
    
    # Show sample filenames
    print(f"\nSample filenames:")
    for f in roi_files[:10]:
        print(f"  {f.name}")

# Check if ROI coordinates are 2D or if there's z-information
print("\n\n=== Checking coordinate dimensions ===")
roi = ImagejRoi.fromfile(roi_files[0])
coords = roi.coordinates()
print(f"Coordinate shape: {coords.shape}")
print(f"Coordinates are 2D (x,y): {coords.shape[1] == 2}")

# Check ROI metadata for stack information
print("\n=== ROI Metadata ===")
roi = ImagejRoi.fromfile(roi_files[0])
print("Available attributes:")
print('\n'.join([attr for attr in dir(roi) if not attr.startswith('_')]))

Found 329 ROI files

=== Checking first few ROI files ===

0212-0230-0195.roi:
  Filename numbers: (212, 230, 195)
  ROI position (z/c/t): 212
  ROI type: 7
  Coordinate bounds: x=[192, 197], y=[227, 232]
  Z position in ROI: 0
  Channel position: 0
  Time position: 0

0213-0229-0194.roi:
  Filename numbers: (213, 229, 194)
  ROI position (z/c/t): 213
  ROI type: 8
  Coordinate bounds: x=[191, 198], y=[226, 233]
  Z position in ROI: 0
  Channel position: 0
  Time position: 0

0214-0229-0194.roi:
  Filename numbers: (214, 229, 194)
  ROI position (z/c/t): 214
  ROI type: 8
  Coordinate bounds: x=[190, 198], y=[225, 234]
  Z position in ROI: 0
  Channel position: 0
  Time position: 0

0215-0229-0193.roi:
  Filename numbers: (215, 229, 193)
  ROI position (z/c/t): 215
  ROI type: 8
  Coordinate bounds: x=[188, 199], y=[224, 235]
  Z position in ROI: 0
  Channel position: 0
  Time position: 0

0216-0229-0193.roi:
  Filename numbers: (216, 229, 193)
  ROI position (z/c/t): 216
  ROI type: 8